In [1]:
import stravalib
import logging
from xml.dom import minidom
import dateutil
import numpy as np

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [2]:
#Global Variables - put your data in the file 'client.secret' and separate the fields with a comma!
client = stravalib.client.Client()
access_token = 'e3ccedc91fceef32077fbb31fc44676446d14bdd'

client.access_token = access_token
athlete = client.get_athlete()

In [3]:
auth_url= client.authorization_url(client_id=19435, redirect_uri='http://localhost:8282/authorized', approval_prompt=u'auto', scope='view_private,write', state=None)
from IPython.core.display import display, HTML
display(HTML("""<a href="{}">{}</a>""".format(auth_url,auth_url)))

In [4]:
code = '6d057263b427852b0489af26e921f8fd25a78852'
access_token = client.exchange_code_for_token(client_id=19435, client_secret='45b776d5beceeb34c290b8a56bf9829d6d4ea5d7', code=code)


In [5]:
client = stravalib.client.Client(access_token=access_token)
athlete = client.get_athlete()
print('athlete name %s, athlete id %s.' %(athlete.firstname, athlete.id))


athlete name Brian, athlete id 3360678.


In [6]:
import datetime
lap_start_date = datetime.datetime(2017, 9, 28)
lap_start_date

datetime.datetime(2017, 9, 28, 0, 0)

In [7]:
runs = []
for activity in client.get_activities(after=lap_start_date):
    if 'ltimate' in activity.name and activity.type == 'Run':
        print(activity.name, activity.type, activity.start_date)
        runs.append(activity)

Ultimate Run 2017-09-29 17:16:08+00:00
Ultimate Run 2017-10-11 17:17:28+00:00


In [8]:
for run in runs:
    print(run)
    for lap in runs[1].laps:
        print(lap.elapsed_time)

<Activity id=1207538661 name='Ultimate' resource_state=2>
0:19:06
0:02:25
0:01:37
0:03:59
0:00:29
0:02:36
0:01:25
0:00:59
0:01:26
0:00:01
0:00:01
0:01:49
0:04:01
0:01:30
0:01:16
0:00:11
0:00:02
<Activity id=1225759439 name='Ultimate' resource_state=2>
0:19:06
0:02:25
0:01:37
0:03:59
0:00:29
0:02:36
0:01:25
0:00:59
0:01:26
0:00:01
0:00:01
0:01:49
0:04:01
0:01:30
0:01:16
0:00:11
0:00:02


In [9]:
lap

<ActivityLap id=3810821033 name='Lap 33' resource_state=2>

In [10]:

def xml_to_float(element):
    if element is not None:
        return float(element[0].firstChild.data)
    else:
        return np.nan
        
def xml_to_time(element):
    if element is not None:
        return datetime.datetime.strptime(element[0].firstChild.data, '%Y-%m-%dT%H:%M:%S.%fZ')
#         return dateutil.parser.parse(element[0].firstChild.data)
    else:
        return np.nan

In [47]:
class Dummy(object):
    pass

self = Dummy()

filename = 'activity_2088811850.tcx'   
       
    
    
doc = minidom.parse(filename)
doc.normalize()
self.tcx = doc.documentElement   

points = []

lap_starts = []
for lap in self.tcx.getElementsByTagName('Lap'):
    lap_starts.append(lap.attributes.items()[0][1])
    
lap_starts = [datetime.datetime.strptime(t, '%Y-%m-%dT%H:%M:%S.000Z') for t in lap_starts]
lap_starts = lap_starts[1:] # Drop first lap (start of day)


#     for track in lap.getElementsByTagName('Track'):
#         for trackpoint in track.getElementsByTagName('Trackpoint'):     

#             _pos = trackpoint.getElementsByTagName('Position') or None
#             if _pos:
#                 _lat = _pos[0].getElementsByTagName('LatitudeDegrees')
#                 _lon = _pos[0].getElementsByTagName('LongitudeDegrees')
#             else:
#                 _lat = None
#                 _lon = None

#             _time = trackpoint.getElementsByTagName('Time')
#             _dist = trackpoint.getElementsByTagName('DistanceMeters') or None
#             _alt = trackpoint.getElementsByTagName('AltitudeMeters') or None

#             _hr_cont = trackpoint.getElementsByTagName('HeartRateBpm') or None
#             if _hr_cont:
#                 _hr = _hr_cont[0].getElementsByTagName('Value') or None
#             else:
#                 _hr = None

#             _cad = trackpoint.getElementsByTagName('Cadence') or None



#             _point = {'lat': xml_to_float(_lat),
#                       'lon': xml_to_float(_lon),
#                       'time': xml_to_time(_time),
#                       'alt':  xml_to_float(_alt),
#                       'hr':   xml_to_float(_hr),
#                       'cad':  xml_to_float(_cad),
#                       'dist': xml_to_float(_dist),
#                      }
#             points.append(_point)

# self.points = points
# # self.activity_type = self.tcx.getElementsByTagName('Activity')[0].attributes.getNamedItem('Sport')
# self.activity_type = 'Biking'

In [49]:
press_delta = datetime.timedelta(0,2)

presses = 1

events = []

for n in range(len(lap_starts) - 1):
    
    
    time = lap_starts[n]
    
    dt = lap_starts[n+1] - lap_starts[n]
    
    # print(presses, time, dt, dt > press_delta)
    
    
    if dt > press_delta:
        events.append([presses, time])
        presses = 1
        
    else:        
        presses += 1
        
events.append([presses, time])

events

[[4, datetime.datetime(2017, 10, 11, 17, 36, 39)],
 [1, datetime.datetime(2017, 10, 11, 17, 39, 5)],
 [1, datetime.datetime(2017, 10, 11, 17, 40, 43)],
 [2, datetime.datetime(2017, 10, 11, 17, 44, 43)],
 [2, datetime.datetime(2017, 10, 11, 17, 45, 14)],
 [3, datetime.datetime(2017, 10, 11, 17, 47, 53)],
 [2, datetime.datetime(2017, 10, 11, 17, 49, 20)],
 [2, datetime.datetime(2017, 10, 11, 17, 50, 20)],
 [2, datetime.datetime(2017, 10, 11, 17, 51, 45)],
 [4, datetime.datetime(2017, 10, 11, 17, 51, 53)],
 [2, datetime.datetime(2017, 10, 11, 17, 53, 44)],
 [1, datetime.datetime(2017, 10, 11, 17, 57, 45)],
 [1, datetime.datetime(2017, 10, 11, 17, 59, 16)],
 [1, datetime.datetime(2017, 10, 11, 18, 0, 32)],
 [4, datetime.datetime(2017, 10, 11, 18, 0, 46)]]

In [50]:
events

[[4, datetime.datetime(2017, 10, 11, 17, 36, 39)],
 [1, datetime.datetime(2017, 10, 11, 17, 39, 5)],
 [1, datetime.datetime(2017, 10, 11, 17, 40, 43)],
 [2, datetime.datetime(2017, 10, 11, 17, 44, 43)],
 [2, datetime.datetime(2017, 10, 11, 17, 45, 14)],
 [3, datetime.datetime(2017, 10, 11, 17, 47, 53)],
 [2, datetime.datetime(2017, 10, 11, 17, 49, 20)],
 [2, datetime.datetime(2017, 10, 11, 17, 50, 20)],
 [2, datetime.datetime(2017, 10, 11, 17, 51, 45)],
 [4, datetime.datetime(2017, 10, 11, 17, 51, 53)],
 [2, datetime.datetime(2017, 10, 11, 17, 53, 44)],
 [1, datetime.datetime(2017, 10, 11, 17, 57, 45)],
 [1, datetime.datetime(2017, 10, 11, 17, 59, 16)],
 [1, datetime.datetime(2017, 10, 11, 18, 0, 32)],
 [4, datetime.datetime(2017, 10, 11, 18, 0, 46)]]

In [55]:
event_lookup = [
    '',
    'team_point',
    'opponent_point',
    'my_point',
    'game',
]

In [56]:
games = []

base_game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0}
game = base_game
for event in events:
    game['end_time'] = event[1]

    event_type = event_lookup[event[0]]
    if event_type == 'game':
        games.append(game)
        game = {'my_point': 0, 'team_point': 0, 'opponent_point': 0}
        
    elif event_type == 'my_point':
        game[event_type] += 1
        game['team_point'] += 1
    else:
        game[event_type] += 1

games.append(game)
games

[{'end_time': datetime.datetime(2017, 10, 11, 17, 36, 39),
  'my_point': 0,
  'opponent_point': 0,
  'team_point': 0},
 {'end_time': datetime.datetime(2017, 10, 11, 17, 51, 53),
  'my_point': 1,
  'opponent_point': 5,
  'team_point': 3},
 {'end_time': datetime.datetime(2017, 10, 11, 18, 0, 46),
  'my_point': 0,
  'opponent_point': 1,
  'team_point': 3},
 {'my_point': 0, 'opponent_point': 0, 'team_point': 0}]

In [57]:
import pandas as pd

In [58]:
pd.DataFrame(games)

,end_time,my_point,opponent_point,team_point
0,2017-10-11 17:36:39,0,0,0
1,2017-10-11 17:51:53,1,5,3
2,2017-10-11 18:00:46,0,1,3
3,NaT,0,0,0
